In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
image_dir_path='/content/drive/MyDrive/covid-19/covid_dataset'
data_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [4]:
training_data=data_generator.flow_from_directory(
    image_dir_path,
    seed=42,
    target_size=(200,200),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Validation_data=data_generator.flow_from_directory(
    image_dir_path,
    seed=42,
    target_size=(200,200),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 76 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [11]:
#using tensorflow sequential model
seq_model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])
seq_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [13]:
trained_seq_model=seq_model.fit_generator(
    training_data,
    epochs=15,
    validation_data=Validation_data)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
3/3 [==============================] - 8s 2s/step - loss: 0.2844 - accuracy: 0.8816 - val_loss: 0.1280 - val_accuracy: 1.0000
Epoch 2/15
3/3 [==============================] - 8s 2s/step - loss: 0.1208 - accuracy: 0.9605 - val_loss: 0.0592 - val_accuracy: 1.0000
Epoch 3/15
3/3 [==============================] - 8s 2s/step - loss: 0.1367 - accuracy: 0.9605 - val_loss: 0.1014 - val_accuracy: 0.9444
Epoch 4/15
3/3 [==============================] - 8s 3s/step - loss: 0.1748 - accuracy: 0.9605 - val_loss: 0.1346 - val_accuracy: 0.8889
Epoch 5/15
3/3 [==============================] - 8s 2s/step - loss: 0.1239 - accuracy: 0.9342 - val_loss: 0.0866 - val_accuracy: 0.9444
Epoch 6/15
3/3 [==============================] - 8s 3s/step - loss: 0.0758 - accuracy: 0.9868 - val_loss: 0.1064 - val_accuracy: 0.9444
Epoch 7/15
3/3 [==============================] - 8s 2s/step - loss: 0.1150 - accuracy: 0.9737 - val_loss: 0.0706 - val_accuracy: 1.0000
Epoch 8/15
3/3 [=========================

In [14]:
seq_model.save('covid_classifier')

INFO:tensorflow:Assets written to: covid_classifier/assets


In [15]:
import numpy as np
test_image=tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/covid-19/covid_dataset/covid/1-s2.0-S0929664620300449-gr2_lrg-a.jpg',target_size=(200,200,3))
test_image=tf.keras.preprocessing.image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=seq_model.predict(test_image)
if result[0][0]==1:
    print('Tested Positive for COVID')
else:
    print('Tested Negative For COVID')

Tested Positive for COVID
